<a href="https://colab.research.google.com/github/hhumayune/Fake-News-Detector/blob/master/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import re

# importing datasets
true_path = '/content/drive/MyDrive/News-_dataset/True.csv'
fake_path = '/content/drive/MyDrive/News-_dataset/Fake.csv'

true_df = pd.read_csv(true_path)
fake_df = pd.read_csv(fake_path)

In [11]:
# briefly visualizing the datasets
display(true_df)
display(fake_df)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [12]:
# Data preprocessing part
# Data combination
# Adding Labels to both df
true_df['true'] = 1
fake_df['true'] = 0
# Concat. this basically adds true_df and fake_df
df = pd.concat([true_df, fake_df])
# Shuffling the data
from sklearn.utils import shuffle
df = shuffle(df).reset_index(drop=True)
# displaying again
display(df)


,title,text,subject,date,true
0,"Northern Ireland won't stay in EU market, cust...",BRUSSELS (Reuters) - Northern Ireland will not...,worldnews,"November 6, 2017",1
1,#VeryFakeNewsCNN BUSTED After Getting MAJOR De...,Would CNN even know the truth if it bit them i...,politics,"Dec 8, 2017",0
2,"Tired Of Bigotry, California Enacts Travel Ba...",Donald Trump keeps trying and failing to enact...,News,"June 23, 2017",0
3,How Is Panama’s “Migrant Crisis” Giving A FREE...,Let this sink in The word has been out for som...,politics,"Aug 10, 2016",0
4,"Guatemala to move embassy to Jerusalem, backin...",GUATEMALA CITY (Reuters) - Guatemalan Presiden...,worldnews,"December 24, 2017",1
...,...,...,...,...,...
44893,"Uber, Grab to tighten up on deliveries in Phil...",MANILA (Reuters) - Ride-hailing platforms Uber...,worldnews,"September 29, 2017",1
44894,BOOM! CUSTOMS AGENTS IGNORE Obama Appointed Ju...,"On Friday, a federal judge in New York issued ...",left-news,"Jan 30, 2017",0
44895,White House says Iran's progress on nuclear de...,WASHINGTON (Reuters) - Iran has made important...,politicsNews,"January 15, 2016",1
44896,Conservatives Desperately Turn To Social Medi...,"Early Sunday morning, Omar Mateen walked into ...",News,"June 12, 2016",0


In [13]:
# Preprocessing functions for eliminating noise
def preprocess_text(text):
    # Replace characters that are not between a to z or A to Z with whitespace
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert all characters into lower-case ones. This doesnt really matter if you use BERT
    text = text.lower()
    # Remove inflectional morphemes like "ed", "est", "s", and "ing" from their token stem
    text = re.sub('(ed|est|s|ing)$', '', text)
    return text

# Apply preprocessing to the title column
df['title'] = df['title'].apply(preprocess_text)
display(df)

,title,text,subject,date,true
0,northern ireland won t stay in eu market cust...,BRUSSELS (Reuters) - Northern Ireland will not...,worldnews,"November 6, 2017",1
1,veryfakenewscnn busted after getting major de...,Would CNN even know the truth if it bit them i...,politics,"Dec 8, 2017",0
2,tired of bigotry california enacts travel ba...,Donald Trump keeps trying and failing to enact...,News,"June 23, 2017",0
3,how is panama s migrant crisis giving a free...,Let this sink in The word has been out for som...,politics,"Aug 10, 2016",0
4,guatemala to move embassy to jerusalem backin...,GUATEMALA CITY (Reuters) - Guatemalan Presiden...,worldnews,"December 24, 2017",1
...,...,...,...,...,...
44893,uber grab to tighten up on deliveries in phil...,MANILA (Reuters) - Ride-hailing platforms Uber...,worldnews,"September 29, 2017",1
44894,boom customs agents ignore obama appointed ju...,"On Friday, a federal judge in New York issued ...",left-news,"Jan 30, 2017",0
44895,white house says iran s progress on nuclear de...,WASHINGTON (Reuters) - Iran has made important...,politicsNews,"January 15, 2016",1
44896,conservatives desperately turn to social medi...,"Early Sunday morning, Omar Mateen walked into ...",News,"June 12, 2016",0


In [14]:
#  Splitting the Dataset into Training, Validation, Test 

train_val_df = df.sample(frac = 0.8)
test_df = df.drop(train_val_df.index)

train_df = train_val_df.sample(frac = 0.8)
val_df = train_val_df.drop(train_df.index)

# Reset Index
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print('trainset size:', train_df.shape)
print('valset size:', val_df.shape)
print('testset size:', test_df.shape)
     

trainset size: (28734, 5)
valset size: (7184, 5)
testset size: (8980, 5)


In [15]:
#for IMBALANCED DATA NOT NEEDED
# #shrinking data into 1/10th of the original file
# train_data = 1/10
# val_data = 1/10
# test_data= 1/10

# train_true = train_df[train_df['true'] == 1]
# train_fake = train_df[train_df['true'] == 0]
# train_fake = train_fake.iloc[:round(len(train_fake)*train_data),:]
# train_true_len = len(train_true)
# train_fake_len = len(train_fake)
# train_df = pd.concat([train_true, train_fake])
# train_df = shuffle(train_df).reset_index(drop=True)

# val_true = val_df[val_df['true'] == 1]
# val_fake = val_df[val_df['true'] == 0]
# val_fake = val_fake.iloc[:round(len(val_fake)*val_data),:]
# val_true_len = len(val_true)
# val_fake_len = len(val_fake)
# val_df = pd.concat([val_true, val_fake])
# val_df = shuffle(val_df).reset_index(drop=True)
# test_true = test_df[test_df['true'] == 1]
# test_fake = test_df[test_df['true'] == 0]
# test_fake = test_fake.iloc[:round(len(test_fake)*test_data),:]
# test_true_len = len(test_true)
# test_fake_len = len(test_fake)
# test_df = pd.concat([test_true, test_fake])
# test_df = shuffle(test_df).reset_index(drop=True)

# len_df = pd.DataFrame({'ind': ['Train', 'Validation', 'Test'], 'True': [train_true_len, val_true_len, test_true_len], 'Fake': [train_fake_len, val_fake_len, test_fake_len]})
# len_df.set_index('ind', inplace=True)
# len_df.index.name = None

# len_df

In [16]:
# dataset to csv
train_df.to_csv('train.tsv', sep='\t', index=False)
val_df.to_csv('val.tsv', sep='\t', index=False)
test_df.to_csv('test.tsv', sep='\t', index=False)

# concat again
df = pd.concat([train_df, val_df, test_df])
display(df)

,title,text,subject,date,true
0,was michelle obama in on beyonce s cop hating ...,In an interview that is a tradition before the...,left-news,"Feb 8, 2016",0
1,u s democratic senator dick durbin undergoes ...,"CHICAGO (Reuters) - U.S. Senator Dick Durbin, ...",politicsNews,"May 2, 2017",1
2,god squad jury finds polygamous mormon towns ...,Jamie Ross Courthouse News ServicePHOENIX Tw...,Middle-east,"March 12, 2016",0
3,priceless trump s answer about putin firing u...,This is priceless! President Trump just gave t...,politics,"Aug 10, 2017",0
4,donald trump is an illegitimate president ele...,Russian interference in our election should im...,News,"December 10, 2016",0
...,...,...,...,...,...
8975,former obama adviser blasts trump calls him ...,Donald Trump has helped America s racists feel...,News,"January 10, 2016",0
8976,grab em by the p ssy scott baio says wife...,Donald Trump supporter Scott Baio has filed a ...,News,"December 16, 2016",0
8977,hillary s pastor compares her election loss to...,"A United Methodist minister, Rev. Bill Shillad...",politics,"Aug 11, 2017",0
8978,u s navy secretary says any patriot should ...,WASHINGTON (Reuters) - U.S. Navy Secretary Ric...,politicsNews,"August 11, 2017",1


In [17]:
# Word Embedding
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer

# Creating A Tokenizer To Tokenize The Words And Create Sequences Of Tokenized Words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(train_df['title'])

train_sequences = tokenizer.texts_to_sequences(train_df['title'])
val_sequences = tokenizer.texts_to_sequences(val_df['title'])
test_sequences = tokenizer.texts_to_sequences(test_df['title'])
     
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Adding Padding
padded_train = pad_sequences(train_sequences,maxlen = 42, padding = 'post', truncating = 'post')
padded_val = pad_sequences(val_sequences,maxlen = 42, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 42, padding = 'post', truncating = 'post')

# display(df)
     

NameError: ignored